In [81]:
# coding=utf-8
import os
import itertools
import pandas as pd
import numpy as np

In [82]:
def find_files(root_dir=None, prefix=None):
    file_paths = []
    
    for root, dirs, files in os.walk(root_dir):
        for name in files:
            if name.decode("utf-8").lower().startswith(prefix):
                file_paths.append(os.path.join(root, name))
    return file_paths

In [83]:
def parse_journal(fpath, output_path):
    xls = pd.ExcelFile(fpath)

    # Parse Status sheet
    df = xls.parse(
        sheetname=0, 
        header=0,
        index_col=0,
        parse_cols="B:E"
    )
    
    df.to_csv(
        os.path.join(output_path, 'СТАТУС.csv'), 
        encoding="utf-8", 
        sep="\t"
    )
    
    # Parse Machinetools sheets 
    for sheet_idx in range(1, 13):
        df = xls.parse(
            sheetname=sheet_idx, 
            header=1,
            parse_cols="B:N",
            skiprows=[2]
        )
        
        # Cпособ найти индекс строки, где начинается ненужная часть таблицы
        # -4 нужно для багованных листов, где 00:00 проставлено до конца листа :(
        try:
            max_idx = df.loc[df.isnull().all(axis=1)].index[0]
        except IndexError:
            max_idx = -4
        
        filename = xls.sheet_names[sheet_idx].encode("utf-8") + "_станок.csv"
        df[:max_idx].to_csv(
            os.path.join(output_path, filename), 
            encoding="utf-8",
            sep="\t"
        )

In [84]:
# coding=utf-8
def parse_plan(fpath, output_path):
    xls = pd.ExcelFile(fpath)
    xls.sheet_names

    df = xls.parse(
        sheetname=0,
        header=0,
        index_col=None
    ).dropna(how='all', axis=1)
    
    # ~ old
    # find out num of header rows
    # headers_num = 0
    # for index, row in df.iterrows():
    #     if not np.isnan(index):
    #         break
    #     headers_num += 1

    # max_multiindex_row = df.loc[
    #     df[u"Наименование"].str.contains(u"Заказ", case=False, na=False)
    # ]
    
    df = df[df[u"Наименование"].notnull()].reset_index(drop=False)
    # Create order_num column
    # df.insert(0, column=u"whatever", value=np.nan)
    df = df.reindex(method='ffill')
    
    headers_num = 3
    xx = xls.parse(
        sheetname=0,
        header=None,
        index_col=False,
    )[0:headers_num].dropna(how='all')
    xx.iloc[0,0] = u"Номер детали"
    
    # xx.insert(0, column=u"whatever", value=u"Заказ")
    
    # Fill down
    xx.fillna(method='ffill', axis=0, inplace=True)
    # Fill across
    # xx = xx.fillna(method='ffill', axis=1)

    xx.to_csv(
        os.path.join(output_path, 'multi_index.csv'),
        header=False,
        index=False,
        encoding="utf-8",
    )
    mxx = pd.read_csv(
        os.path.join(output_path, 'multi_index.csv'), 
        header=[0,1,2],
        skipinitialspace=True,
        tupleize_cols=True,
        encoding="utf-8",
    )
    df.columns = pd.MultiIndex.from_tuples(mxx.columns)
    
    df.to_csv(
        os.path.join(output_path, "parsed_План.csv"), 
        encoding="utf-8",
        sep="\t"
    )
    return df.head()
    
    # dff = xls.parse(
    #     sheetname=0,
    #     header=[0,1,2],
    #     index_col=None
    # )
    # dff = dff.reindex(method='ffill')
    # dff.fillna(method='ffill', axis=0, inplace=True)
    # dff = dff.fillna(method='ffill', axis=1)
    # ~

    # # Headers as rows for now
    # df = xls.parse(sheetname=0, header=None, index_col=None)
    # 
    # # Create order_num column
    # #df.insert(0, column=u"whatever", value=np.nan)
    # 
    # headers = df.iloc[:headers_num]
    # 
    # # Fill down
    # headers = headers.fillna(method='ffill')
    # # fill few column indicies (fucked up indexing)
    # headers.iloc[0,0] = u"Номер заказа"
    # headers.iloc[0,1] = u"Номер детали"
    # 
    # # Fill across (carefully)
    # headers_not_to_fill = headers.iloc[:,:5]
    # 
    # headers_to_fill = headers.iloc[:,5:]
    # headers = pd.concat([
    #     headers_not_to_fill,
    #     headers_to_fill.fillna(method='ffill', axis=1)
    # ], axis=1)
    # 
    # df = df.iloc[headers_num:]
    # df = df.reset_index(drop=True)

    # Create multiindex column names
    # df.columns = pd.MultiIndex.from_arrays(mxx.values.tolist())
    # dup_first_index_dates = df.columns
    # print df.head()
    # blah = df[u"Наименование"] #.ix[df[u"Наименование"].notnull()].index.tolist()
    # print blah
    # max_multiindex_row = df.loc[
    #     df[u"Наименование"].str.contains(u"Заказ", case=False, na=False)
    # ]

In [85]:
def parse_journals(fpaths, output_path):
    for xls_path in fpaths:
        print "Processing status sheets: " + str(xls_path)
        xls = pd.ExcelFile(xls_path)
    
        # Parse Status sheet
        df_status = xls.parse(
            sheetname=0, 
            header=0,
            index_col=0,
            parse_cols="B:E"
        )
        
        df_status.to_csv(
            os.path.join(output_path, 'СТАТУС.csv'), 
            encoding="utf-8", 
            sep="\t"
        )
    
    # Parse Machinetools sheets
    machinetool_names = []
    machinetool_frames = []
    for sheet_idx in range(1, 13):
        print "Processing machinetools sheet for each journal: " + str(sheet_idx)
        year_frames = []
        
        for xls_path in fpaths:
            xls = pd.ExcelFile(xls_path)

            df = xls.parse(
                sheetname=sheet_idx,
                header=1,
                parse_cols="B:N",
                skiprows=[2]
            ).dropna(how='all', axis=1).dropna(how='all')
            
            if any(df[u"Время старт (чч:мм)"] == u"Статус"):
                df = df[:-4]
            year_frames.append(df)
            
        machinetool_names.append(xls.sheet_names[sheet_idx])
        machinetool_df = pd.concat(year_frames, ignore_index=True)
        machinetool_frames.append(machinetool_df)
        # filename = xls.sheet_names[sheet_idx].encode("utf-8") + "_станок.csv"
    
    result = pd.concat(
        machinetool_frames, 
        keys=machinetool_names,
        names=[u"Станок", u"Index"]
    )
    result.to_csv(
        os.path.join(output_path, u"Журнал станков за 3 месяца.csv"),
        encoding="utf-8",
        sep="\t"
    )

In [86]:
ROOT_DIR = r"/home/larleyt/fl/ML/VirtualFactory/data collection"
OUTPUT_JOURNALS_PATH = ROOT_DIR + r"/CSVs/Journals"
OUTPUT_PLANS_PATH = ROOT_DIR + r"/CSVs/Plans"

journal_file_paths = find_files(ROOT_DIR, u"новый")
plan_file_paths = find_files(ROOT_DIR, u"план 2012-16")

# parse_journals(journal_file_paths, OUTPUT_JOURNALS_PATH)
parse_plan(plan_file_paths[0], OUTPUT_PLANS_PATH)


# for xls_path in journal_file_paths:
#     print "Processing journals: " + xls_path
#     xls_named_dir = os.path.join(
#         OUTPUT_JOURNALS_PATH, 
#         os.path.basename(xls_path).split(".")[0])
#     print xls_path
#     if not os.path.exists(xls_named_dir):
#         os.mkdir(xls_named_dir)
#     parse_journal(xls_path, xls_named_dir)

,Номер детали,Наименование,срок сдачи,кол-во заказ,изготовлено,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0,заготовка,...,Unnamed: 42_level_0,Отгружено-Приход,Unnamed: 44_level_0,Unnamed: 45_level_0,Unnamed: 46_level_0,Unnamed: 47_level_0,Unnamed: 48_level_0,Unnamed: 49_level_0,Unnamed: 50_level_0,Unnamed: 51_level_0
,Номер детали,Наименование,срок сдачи,кол-во заказ,Станок,Unnamed: 5_level_1,Unnamed: 6_level_1,Unnamed: 7_level_1,Unnamed: 8_level_1,размеры марка,...,Цех 1,Покрытие,Unnamed: 44_level_1,Аксион,Unnamed: 46_level_1,Спектр,Unnamed: 48_level_1,Прочие,Unnamed: 50_level_1,СТМ
,Номер детали,Наименование,срок сдачи,кол-во заказ,кол-во,дата,подг.вр,маш.вр.,ответственный,размеры марка,...,В. Шоссе,Расход,Приход,Расход,Приход,Расход,Приход,Расход,Приход,Расход
0,NaN,Заказ № 11463 от 28.12.2011 500шт. Март 500шт...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,ЦДКТ.731111.001 Корпус,NaN,1000,NaN,NaN,NaN,NaN,NaN,Д16 170x90#45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Заказ № 11461 от 27.11.2011 срок поставки 15....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,ЦДКТ.731123.005 Корпус,2012-02-24 00:00:00,13,14,2012-02-10 00:00:00,NaN,NaN,Шкляев И.Н.,Пруток ДКРНТ D25xL80 ЛС59-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,ЦДКТ.754142.040 Крышка,2012-02-15 00:00:00,13,20,2012-02-08 00:00:00,NaN,NaN,Шкляев И.Н.,"ДПРНМ 0,6 205x305#0,6* зам-ль ДПРНМ 0,5 Л63",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
